In [1]:
import json
import csv

# Path to the COCO annotated JSON file
json_file_path = '/kaggle/input/taco-single-class/data/data/custom_val.json'

# Path to the CSV file you want to create
csv_file_path = 'output.csv'

def convert_coco_to_csv(json_file_path, csv_file_path):
    with open(json_file_path, 'r') as json_file:
        coco_data = json.load(json_file)

    images = coco_data['images']
    annotations = coco_data['annotations']

    # Create a dictionary to store image information by image ID
    image_info = {}
    for image in images:
        image_info[image['id']] = {
            'filename': image['file_name'],
            'width': image['width'],
            'height': image['height']
        }

    # Create a list to store annotation information
    annotation_list = []
    for annotation in annotations:
        image_id = annotation['image_id']
        image_info_entry = image_info.get(image_id)
        if image_info_entry:
            annotation_entry = {
                'image': image_info_entry['filename'],
                'xmin': annotation['bbox'][0],
                'ymin': annotation['bbox'][1],
                'xmax': annotation['bbox'][0] + annotation['bbox'][2],
                'ymax': annotation['bbox'][1] + annotation['bbox'][3]
            }
            annotation_list.append(annotation_entry)

    # Write the data to a CSV file
    with open(csv_file_path, 'w', newline='') as csv_file:
        fieldnames = ['image', 'xmin', 'ymin', 'xmax', 'ymax']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(annotation_list)

convert_coco_to_csv(json_file_path, csv_file_path)

In [2]:
!pip install pycocotools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 7.6 MB/s eta 0:00:00a 0:00:01


In [3]:
import os
import pandas as pd
from PIL import Image
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor, FasterRCNN
from torchvision.transforms import functional as F
from sklearn.metrics import precision_recall_fscore_support
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
# Load the Fast R-CNN model
backbone = resnet_fpn_backbone('resnet101', pretrained=True)
model = FasterRCNN(backbone,num_classes=2)
model.load_state_dict(torch.load('/kaggle/input/taco-model/best_model.pth'))
model.eval()

# Directory containing images
image_dir = '/kaggle/input/taco-single-class/data/data/'

# JSON file containing COCO format annotations
coco_json_file = '/kaggle/input/taco-single-class/data/data/custom_val.json'

# Load COCO annotations from the JSON file
coco_gt = COCO(coco_json_file)

# Create a list to store the results
results = []

# Loop through each annotation in the COCO dataset
for annotation in coco_gt.dataset['annotations']:
    image_id = annotation['image_id']
    image_info = coco_gt.loadImgs(image_id)[0]
    image_filename = os.path.join(image_dir, image_info['file_name'])
    class_id = annotation['category_id']
    ground_truth_box = annotation['bbox']

    # Perform object detection on the current image
    image = Image.open(image_filename)
    image_tensor = F.to_tensor(image)
    with torch.no_grad():
        predictions = model([image_tensor])
        
    # Retrieve the predicted bounding boxes, labels, and scores
    predicted_boxes = predictions[0]['boxes']
    predicted_labels = predictions[0]['labels']
    predicted_scores = predictions[0]['scores']


    # Store the results in COCO format
    for bbox in predicted_boxes:
        results.append({
            'image_id': image_id,
            'category_id': class_id,
            'bbox': [bbox[0].item(), bbox[1].item(), bbox[2].item() - bbox[0].item(), bbox[3].item() - bbox[1].item()],
            'score': 1.0  # You can set a fixed score since you're not calculating confidence scores
        })


# Initialize COCO evaluation
coco_dt = coco_gt.loadRes(results)
coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')

# Run the evaluation
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

# Print the metrics in the specified format
print("IoU metric: bbox")
print(f"Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = {coco_eval.stats[0]:.3f}")
print(f"Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = {coco_eval.stats[1]:.3f}")
print(f"Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = {coco_eval.stats[2]:.3f}")
print(f"Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = {coco_eval.stats[3]:.3f}")
print(f"Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = {coco_eval.stats[4]:.3f}")
print(f"Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = {coco_eval.stats[5]:.3f}")
print(f"Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = {coco_eval.stats[6]:.3f}")
print(f"Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = {coco_eval.stats[7]:.3f}")
print(f"Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = {coco_eval.stats[8]:.3f}")
print(f"Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = {coco_eval.stats[9]:.3f}")
print(f"Average Recall     (AR) @[ IoU=0.50:0.95 | area= medium | maxDets=100 ] = {coco_eval.stats[10]:.3f}")
print(f"Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = {coco_eval.stats[11]:.3f}")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'backbone_name' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weight

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.90s).
Accumulating evaluation results...
DONE (t=0.18s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.020
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.039
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.016
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.008
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.012
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.030
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.212
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.419
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets